In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pyodbc


In [ ]:
input_folder = r'..\Historical Data\Files\dynamic'
output_folder = r'\Historical Data\Files\dynamic-processed'
files = os.listdir(input_folder)

In [ ]:
headers = ['DATE'
,'LOANID'
,'LOANID_OLD'
,'INTRATETYPE'
,'TOTALRATE'
,'BASERATE'
,'INTRATEDES'
,'INTRATEPROD'
,'OPTIONEXER'
,'BALANCE'
,'DRAWDOWN'
,'TECHNICALBALANCE'
,'ARREARS'
,'DPD'
,'PRINCIPAL_DUE'
,'INTEREST_DUE'
,'PRINCIPAL_RECEIVED'
,'INTEREST_RECEIVED'
,'ONEOFFPAYMENT'
,'ONEOFFPAYMENT_TYPE'
,'OTHERDUE'
,'OTHERDUE_RECEIVED'
,'WRITEOFF'
,'NPL_FLAG'
,'CLOSURE_TYPE'
,'UNSECURED_FLAG'
,'SOLD_FLAG'
,'LOANSTATUS'
,'LOANCLOSEDATE']

numeric_cols = ['TOTALRATE'
 ,'BASERATE'
 ,'BALANCE'
 ,'DRAWDOWN'
 ,'TECHNICALBALANCE'
 ,'ARREARS'
 ,'DPD'
 ,'PRINCIPAL_DUE'
 ,'INTEREST_DUE'
 ,'PRINCIPAL_RECEIVED'
 ,'INTEREST_RECEIVED'
 ,'ONEOFFPAYMENT'
 ,'WRITEOFF'
 ,'OTHERDUE'
 ,'OTHERDUE_RECEIVED'
 ,'ONEOFFPAYMENT_TYPE'
]

cols_strs = [col for col in headers if 'DATE' not in col]

In [ ]:
def splitline(line,sep=';'):
    return line.split(';')
def handle_date(series):
    return pd.to_datetime(series.astype(float).replace(0,np.nan).astype(str),format='%Y%m%d')
def handle_str(series):
    return series.str.strip()
def handle_numeric(series):
#    return series.str.replace("+",'').str.replace("-",'').str.replace(",",'.').astype(float)
    return series.str.replace(",",'.').astype(float)


In [ ]:
def make_sanitised_tape(lines,headers):

    tape = pd.DataFrame(lines,columns=headers)
    
    tape['DATE'] = handle_date(tape['DATE'])
    tape['LOANCLOSEDATE'] = handle_date(tape['LOANCLOSEDATE'])
    for col in cols_strs:
        tape[col] = handle_str(tape[col])
        
    for col in numeric_cols:
        tape[col] = handle_numeric(tape[col])

    additional_data = tape['LOANID'].str.split('-',expand=True).filter([1,2,3,4])
    
    additional_data.columns = ['PRODUCTID','BRANCHID','BASELOANID','SUBACCOUNT']
    
    additional_data = additional_data.astype(float)
        
    tape = pd.concat([tape,additional_data],axis=1)
    
    return tape
    

In [ ]:
lines_count = 0
for file_name in files[:]:
    lines = []
    print(f'Started Working on file :{file_name}')
    with open(os.path.join(input_folder,file_name),'r') as doc:
        for index,line in enumerate(doc):
            if index==0:
                continue
            split_line= splitline(line)[:29]
            lines.append(split_line)
    lines_count = lines_count + len(lines)
    print(f'# Lines Completed, {lines_count}')
    print(f'Reading Completed , Processing Started')
    tape = make_sanitised_tape(lines=lines,headers=headers)
    tape.to_csv(os.path.join(output_folder,file_name),index=False)
    print(f'{file_name} sanitised version saved.')

In [ ]:
# For static files

In [ ]:
path = r"C:\Users\shivampundir\Downloads\Peninsula March 23\May 23 Cutoff\Historical Data\Files\20120731-20230531-STATIC.CSV"
f = open(path,'r')

first_line = f.readline()

headers = first_line.split(";")[:15]

lines = f.readlines()

values = []
for line in lines:
    values.append(line.split(';')[:15])

In [ ]:
static_data = pd.DataFrame(values,columns=headers)

In [ ]:
static_data['Origination Date'] = pd.to_datetime(static_data['Origination Date'],format='%Y%m%d',errors='coerce')
static_data['Maturity Date']    = pd.to_datetime(static_data['Maturity Date'].replace('00000000',np.nan),format='%Y%m%d',errors='coerce')


static_data['Purpose'] = static_data['Purpose'].str.strip()

static_data['Original LTV'] = handle_numeric(static_data['Original LTV'])

static_data['Original Principal Balance'] = handle_numeric(static_data['Original Principal Balance'])

cols = ['Loan ID (current loan)',
 'Loan ID (historical loan)',
 'Purpose',
 'Product',
 'Currency Denomination',
 'Interest Rate Type',
 'Amortisation Type',
 'Optionality',
 'CHF Loan Flag',
 'Property Region']
# 'Loan status']

static_data.loc[:,cols] = static_data.loc[:,cols].astype(str)

In [ ]:
static_data['Loan ID (current loan)'] = handle_str(static_data['Loan ID (current loan)'])
static_data['Loan ID (historical loan)'] = handle_str(static_data['Loan ID (historical loan)'])

In [ ]:
additional_data = static_data['Loan ID (current loan)'].str.split('-',expand=True).filter([1,2,3,4])
    
additional_data.columns = ['PRODUCTID','BRANCHID','BASELOANID','SUBACCOUNT']

In [ ]:
additional_data_old = static_data['Loan ID (historical loan)'].str.split('-',expand=True).filter([1,2,3,4])
    
additional_data_old.columns = ['PRODUCTID_OLD','BRANCHID_OLD','BASELOANID_OLD','SUBACCOUNT_OLD']

In [ ]:
additional_data = additional_data.astype(float)
additional_data_old = additional_data_old.astype(float)
        
static_data = pd.concat([static_data,additional_data,additional_data_old],axis=1)

In [ ]:
static_data.to_excel(r"\- Closed Loans Tape.xlsx")

In [ ]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    driver='{ODBC Driver 17 for SQL Server}',
    server='localhost',
    database='Peninsula',
    trusted_connection='yes'
)

In [ ]:
def get_data_for_query():
    query = "select Date,BaseLoanId	, LoanID, LoanId_Old , INTRATETYPE, TOTALRATE , BASERATE,INTRATEDES,BALANCE,DRAWDOWN,ARREARS,DPD,PRINCIPAL_DUE,INTEREST_DUE,OTHERDUE,PRINCIPAL_RECEIVED,INTEREST_RECEIVED,OTHERDUE_RECEIVED,ONEOFFPAYMENT,WRITEOFF,NPL_FLAG,CLOSURE_TYPE,LOANCLOSEDATE from dynamic where PRODUCTID = 6 and Date > '2023-03-31'"
    df = pd.read_sql(query,conn)
    return df

def insert_data_for_query():
    #TODO
    return


In [ ]:
df = get_data_for_query()